# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [10]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [11]:
name = "high_vs_random_min_len_1000"
file_name = f"csv/clean_{name}.csv"
df = pd.read_csv(file_name)
df

,Unnamed: 0,text_clean,label
0,0,simplepasscodebuttons not working on ios 1131 ...,1
1,1,pedroborgeskirbymetatagsv210 2 vulnerabilities...,1
2,2,when i use thrift protocol the classes generat...,0
3,3,the tiled gallerys visible columns in the edit...,0
4,4,core chinese characters not displaying in work...,1
...,...,...,...
76415,76415,bug sidebar statistics fail to load in appon f...,0
76416,76416,ena consensus submissions x instituteenaassemb...,0
76417,76417,nearestneighbor upsampling regression bug whe...,0
76418,76418,cve202122880 high detected in rails523gem acti...,1


In [12]:
# drop Unnamed: 0
df = df.drop(columns=["Unnamed: 0"])
df

,text_clean,label
0,simplepasscodebuttons not working on ios 1131 ...,1
1,pedroborgeskirbymetatagsv210 2 vulnerabilities...,1
2,when i use thrift protocol the classes generat...,0
3,the tiled gallerys visible columns in the edit...,0
4,core chinese characters not displaying in work...,1
...,...,...
76415,bug sidebar statistics fail to load in appon f...,0
76416,ena consensus submissions x instituteenaassemb...,0
76417,nearestneighbor upsampling regression bug whe...,0
76418,cve202122880 high detected in rails523gem acti...,1


In [13]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
1,38265
0,38155


In [14]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [15]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [16]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 7642
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 61136
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 7642
    })
})

In [17]:
# Uncomment this if not logged in.
#notebook_login()

In [18]:
# Push to Hugging Face Hub
ds.push_to_hub(name)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/62 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]